In [1]:
import os
import ee
import geemap
import ee
from google.colab import auth

In [3]:
# Authenticate and initialize the Earth Engine session
auth.authenticate_user()
ee.Initialize(project='amin-428605')

In [12]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# Define the area of interest (replace with your specific coordinates)
aoi = ee.Geometry.Rectangle([100.0, 1.0, 101.0, 2.0])

# Load the Hansen Global Forest Change dataset
gfc = ee.Image('UMD/hansen/global_forest_change_2019_v1_7')

# Extract the forest cover for the year 2000
forestCover2000 = gfc.select(['treecover2000'])

# Extract the forest loss year band
lossYear = gfc.select(['lossyear'])

# Create a mask for changes between 2013 and 2019
changeMask = lossYear.gte(13).And(lossYear.lte(19))

# Apply the mask to the loss year band
changes = lossYear.updateMask(changeMask)

# Remap the years to make them more intuitive (2013 = 13, 2014 = 14, etc.)
changesRemapped = changes.remap(
  [13, 14, 15, 16, 17, 18, 19],
  [2013, 2014, 2015, 2016, 2017, 2018, 2019],
  0
).rename('year')

# Calculate the total area of change
areaImage = changesRemapped.gt(0).multiply(ee.Image.pixelArea())
totalChange = areaImage.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

print('Total area of change (sq m):', totalChange.getInfo()['year'])

# Calculate change per year
years = ee.List.sequence(2013, 2019)
yearlyChange = years.map(lambda year: {
    'year': year,
    'area': changesRemapped.eq(ee.Number(year)).multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=30,
        maxPixels=1e9
    ).get('year')
})

print('Yearly change:', yearlyChange.getInfo())

# Load Sentinel-2 imagery for RGB composite
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate('2019-01-01', '2019-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median() \
    .clip(aoi)

# Visualization parameters for RGB composite
rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

# Visualization setup for forest change
visParams = {
    'min': 2013,
    'max': 2019,
    'palette': ['ff0000', 'ff3300', 'ff6600', 'ff9900', 'ffcc00', 'ffff00', 'ccff00']
}

# Initialize the map
Map = geemap.Map(center=[1.5, 100.5], zoom=8)

# Add the forest cover layer (year 2000)
Map.addLayer(forestCover2000.clip(aoi), {'palette': ['000000', '00FF00'], 'max': 100}, 'Forest Cover 2000')

# Add the changes layer (2013-2019)
Map.addLayer(changesRemapped.clip(aoi), visParams, 'Forest Loss Year')

# Add the RGB composite layer
Map.addLayer(sentinel2, rgbVis, 'Sentinel-2 RGB')

# Display the map
Map


Total area of change (sq m): 1442201611.5062702
Yearly change: [{'area': 294073174.25911885, 'year': 2013}, {'area': 253410912.25766194, 'year': 2014}, {'area': 131037076.50903176, 'year': 2015}, {'area': 246660821.3092675, 'year': 2016}, {'area': 200922277.48114657, 'year': 2017}, {'area': 183605818.4017925, 'year': 2018}, {'area': 132491531.28825086, 'year': 2019}]


Map(center=[1.5, 100.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [6]:
# Uncomment the following line to install geemap if needed.
# !pip install geemap

import ee
import geemap.foliumap as geemap

# Initialize the Earth Engine API
ee.Initialize()

# Define the area of interest (replace with your specific coordinates)
aoi = ee.Geometry.Rectangle([100.0, 1.0, 101.0, 2.0])

# Load the Hansen Global Forest Change dataset
gfc = ee.Image('UMD/hansen/global_forest_change_2019_v1_7')

# Extract the forest cover for the year 2000
forestCover2000 = gfc.select(['treecover2000'])

# Extract the forest loss year band
lossYear = gfc.select(['lossyear'])

# Create a mask for changes between 2013 and 2019
changeMask = lossYear.gte(13).And(lossYear.lte(19))

# Apply the mask to the loss year band
changes = lossYear.updateMask(changeMask)

# Remap the years to make them more intuitive (2013 = 13, 2014 = 14, etc.)
changesRemapped = changes.remap(
  [13, 14, 15, 16, 17, 18, 19],
  [2013, 2014, 2015, 2016, 2017, 2018, 2019],
  0
).rename('year')

# Visualization parameters for forest change
visParams = {
    'min': 2013,
    'max': 2019,
    'palette': ['ff0000', 'ff3300', 'ff6600', 'ff9900', 'ffcc00', 'ffff00', 'ccff00']
}

# Load Sentinel-2 imagery for RGB composite
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate('2019-01-01', '2019-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median() \
    .clip(aoi)

# Visualization parameters for RGB composite
rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

# Create the map with a split panel
Map = geemap.Map(center=[1.5, 100.5], zoom=8)

# Create Earth Engine layers for the split panel
left_layer = geemap.ee_tile_layer(sentinel2, rgbVis, 'Sentinel-2 RGB')
right_layer = geemap.ee_tile_layer(changesRemapped.clip(aoi), visParams, 'Forest Loss Year')

# Set up the split map with the RGB composite on the left and forest change on the right
Map.split_map(left_layer=left_layer, right_layer=right_layer)

# Display the map
Map


https://geemap.org/notebooks/04_split_panel_map/

# Time;laps 1

In [9]:

import geemap
import ee
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from geemap import cartoee  # Ensure this import is correct

ModuleNotFoundError: No module named 'cartopy'

In [10]:
import os
import ee
import geemap
import ipywidgets as widgets
from PIL import Image

# Initialize the Earth Engine API
ee.Initialize()

# Create a map object
Map = geemap.Map()

# Display the map
Map

# Set output directory for GIF
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Widget styles
style = {"description_width": "initial"}

# Timelapse title input
title = widgets.Text(
    description="Title:", value="Forest Change Timelapse", style=style
)

# Year selection sliders
start_year = widgets.IntSlider(
    description="Start Year:", value=2000, min=1984, max=2021, style=style
)
end_year = widgets.IntSlider(
    description="End Year:", value=2021, min=1984, max=2021, style=style
)

# Animation speed (frames per second)
speed = widgets.IntSlider(
    description="Frames per second:", value=10, min=1, max=30, style=style
)

# Option to download the GIF
download = widgets.Checkbox(value=False, description="Download the GIF", style=style)

# Font settings for the title and progress bar
font_size = widgets.IntSlider(
    description="Font size:", value=30, min=10, max=50, style=style
)
font_color = widgets.ColorPicker(
    concise=False, description="Font color:", value="white", style=style
)
progress_bar_color = widgets.ColorPicker(
    concise=False, description="Progress bar color:", value="blue", style=style
)

# Submit button
submit = widgets.Button(
    description="Submit",
    button_style="primary",
    tooltip="Click to create timelapse",
    style=style,
)

# Output area for displaying results
output = widgets.Output()

# Function to handle the timelapse creation
def submit_clicked(b):
    with output:
        output.clear_output()
        if start_year.value >= end_year.value:
            print("The end year must be greater than the start year.")
            return
        print("Computing...")

        # Define the area of interest (replace with your specific coordinates)
        aoi = ee.Geometry.Rectangle([100.0, 1.0, 101.0, 2.0])

        # Load the Landsat 8 Surface Reflectance image collection
        def getNDVI(image):
            ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
            return ndvi

        def getForestChange(year):
            start_date = ee.Date.fromYMD(year, 1, 1)
            end_date = ee.Date.fromYMD(year, 12, 31)
            image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                .filterBounds(aoi) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUD_COVER', 30)) \
                .map(getNDVI) \
                .median()
            return image.visualize(min=0, max=1, palette=['white', 'green']).set({'year': year})

        # Generate an ImageCollection for the selected years
        years = ee.List.sequence(start_year.value, end_year.value)
        images = ee.ImageCollection.fromImages(years.map(lambda y: getForestChange(ee.Number(y))))

        # Export images to a temporary directory
        gif_images = []
        for year in years.getInfo():
            image = images.filter(ee.Filter.eq('year', year)).first()
            image_file = os.path.join(out_dir, f"forest_change_{year}.tif")
            geemap.ee_export_image(image, filename=image_file, scale=100, region=aoi)

            # Convert the .tif file to .png for the GIF
            with Image.open(image_file) as img:
                png_file = image_file.replace('.tif', '.png')
                img.save(png_file)
                gif_images.append(Image.open(png_file))

        # Create GIF
        gif_path = os.path.join(out_dir, f"forest_change_timelapse.gif")
        gif_images[0].save(
            gif_path,
            save_all=True,
            append_images=gif_images[1:],
            duration=1000 // speed.value,
            loop=0,
        )

        print(f"GIF saved at {gif_path}")
        if download.value:
            from google.colab import files
            files.download(gif_path)

# Bind the submit button to the handler function
submit.on_click(submit_clicked)

# Display the widgets and output area
widgets.VBox([widgets.HBox([title]),
              widgets.HBox([start_year, end_year]),
              widgets.HBox([speed, download]),
              widgets.HBox([font_size, font_color, progress_bar_color]),
              submit,
              output])


# Editor

In [ ]:
// Define the area of interest (replace with your specific coordinates)
var aoi = ee.Geometry.Rectangle([100.0, 1.0, 101.0, 2.0]);

// Load the Hansen Global Forest Change dataset
var gfc = ee.Image('UMD/hansen/global_forest_change_2019_v1_7');

// Extract the forest cover for the year 2000
var forestCover2000 = gfc.select(['treecover2000']);

// Extract the forest loss year band
var lossYear = gfc.select(['lossyear']);

// Create a mask for changes between 2013 and 2019 (lossYear 13-19 correspond to years 2013-2019)
var changeMask = lossYear.gte(13).and(lossYear.lte(19));

// Apply the mask to the loss year band
var changes = lossYear.updateMask(changeMask);

// Remap the years to make them more intuitive (2013 = 1, 2014 = 2, etc.)
var changesRemapped = changes.remap(
  [13, 14, 15, 16, 17, 18, 19],
  [2013, 2014, 2015, 2016, 2017, 2018, 2019],
  0
).rename('year');

// Define a color palette for the years
var palette = ['ff0000', 'ff3300', 'ff6600', 'ff9900', 'ffcc00', 'ffff00', 'ccff00'];

// Create a visualization parameter object
var visParams = {
  min: 2013,
  max: 2019,
  palette: palette
};

// Add the layers to the map
Map.centerObject(aoi, 8);
Map.addLayer(forestCover2000.clip(aoi), {palette: ['000000', '00FF00'], max: 100}, 'Forest Cover 2000');
Map.addLayer(changesRemapped.clip(aoi), visParams, 'Forest Loss Year');

// Calculate the total area of change
var areaImage = changesRemapped.gt(0).multiply(ee.Image.pixelArea());
var totalChange = areaImage.reduceRegion({
  reducer: ee.Reducer.sum(),
  geometry: aoi,
  scale: 30,
  maxPixels: 1e9
});

print('Total area of change (sq m):', totalChange.get('year'));

// Calculate change per year
var years = ee.List.sequence(2013, 2019);
var yearlyChange = years.map(function(year) {
  var yearArea = changesRemapped.eq(ee.Number(year)).multiply(ee.Image.pixelArea());
  var area = yearArea.reduceRegion({
    reducer: ee.Reducer.sum(),
    geometry: aoi,
    scale: 30,
    maxPixels: 1e9
  });
  return ee.Feature(null, {
    'year': year,
    'area': area.get('year')
  });
});

print('Yearly change:', ee.FeatureCollection(yearlyChange));

// Add a legend
var legend = ui.Panel({
  style: {
    position: 'bottom-left',
    padding: '8px 15px'
  }
});

var legendTitle = ui.Label({
  value: 'Forest Loss Year',
  style: {
    fontWeight: 'bold',
    fontSize: '18px',
    margin: '0 0 4px 0',
    padding: '0'
  }
});
legend.add(legendTitle);

var makeRow = function(color, name) {
  var colorBox = ui.Label({
    style: {
      backgroundColor: '#' + color,
      padding: '8px',
      margin: '0 0 4px 0'
    }
  });
  var description = ui.Label({
    value: name,
    style: {margin: '0 0 4px 6px'}
  });
  return ui.Panel({
    widgets: [colorBox, description],
    layout: ui.Panel.Layout.Flow('horizontal')
  });
};

for (var i = 0; i < palette.length; i++) {
  legend.add(makeRow(palette[i], (2013 + i).toString()));
}

Map.add(legend);


# QGIS Code

In [ ]:
import ee
from ee_plugin import Map

# Initialize the Earth Engine API
ee.Initialize()

# Define the area of interest (replace with your specific coordinates)
aoi = ee.Geometry.Rectangle([100.0, 1.0, 101.0, 2.0])

# Load the Hansen Global Forest Change dataset
gfc = ee.Image('UMD/hansen/global_forest_change_2019_v1_7')

# Extract the forest cover for the year 2000
forestCover2000 = gfc.select(['treecover2000'])

# Extract the forest loss year band
lossYear = gfc.select(['lossyear'])

# Create a mask for changes between 2013 and 2019
changeMask = lossYear.gte(13).And(lossYear.lte(19))

# Apply the mask to the loss year band
changes = lossYear.updateMask(changeMask)

# Remap the years to make them more intuitive (2013 = 13, 2014 = 14, etc.)
changesRemapped = changes.remap(
    [13, 14, 15, 16, 17, 18, 19],
    [2013, 2014, 2015, 2016, 2017, 2018, 2019],
    0
).rename('year')

# Calculate the total area of change
areaImage = changesRemapped.gt(0).multiply(ee.Image.pixelArea())
totalChange = areaImage.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

print('Total area of change (sq m):', totalChange.getInfo()['year'])

# Calculate change per year
years = ee.List.sequence(2013, 2019)
yearlyChange = years.map(lambda year: {
    'year': year,
    'area': changesRemapped.eq(ee.Number(year)).multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=30,
        maxPixels=1e9
    ).get('year')
})

print('Yearly change:', yearlyChange.getInfo())

# Visualization setup for forest loss
visParams = {
    'min': 2013,
    'max': 2019,
    'palette': ['ff0000', 'ff3300', 'ff6600', 'ff9900', 'ffcc00', 'ffff00', 'ccff00']
}

# Add the forest cover layer (year 2000)
Map.addLayer(forestCover2000.clip(aoi), {'palette': ['000000', '00FF00'], 'max': 100}, 'Forest Cover 2000')

# Add the changes layer (2013-2019)
Map.addLayer(changesRemapped.clip(aoi), visParams, 'Forest Loss Year')

# Load a Sentinel-2 image collection for RGB visualization
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()  # Get the least cloudy image

# Define visualization parameters for RGB
rgb_vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # Red, Green, Blue bands for true color
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

# Add the RGB image to the QGIS map, ensuring it aligns with the AOI
Map.addLayer(sentinel2.clip(aoi), rgb_vis_params, 'Sentinel-2 RGB')

# Center the map on the area of interest
Map.centerObject(aoi, 8)


ModuleNotFoundError: No module named 'ee_plugin'